In [1]:
'''
Example: Supervised learning with the SVM algorithm.
'''

'\nExample: Supervised learning with the SVM algorithm.\n'

In [2]:
from __future__ import print_function

import numpy as np

import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.compare import Exact, String

In [3]:
'''
Generated as one data set with 10000 records (5000 originals and 5000 duplicates, with one duplicate per original), 
the originals have been split from the duplicates, into dataset4a.csv (containing the 5000 original records) 
and dataset4b.csv (containing the 5000 duplicate records). 
These two data sets can be used for testing linkage procedures.
'''

from recordlinkage.datasets import load_febrl4

In [4]:
# set logging
rl.logging.set_verbosity(rl.logging.INFO)

In [5]:
# load datasets
print('Loading data...')
dfA, dfB, true_links = load_febrl4(return_links=True)
print(len(dfA), 'records in dataset A')
print(len(dfB), 'records in dataset B')
print(len(true_links), 'links between dataset A and B')

Loading data...
5000 records in dataset A
5000 records in dataset B
5000 links between dataset A and B


In [6]:
# start indexing
print('Build index...')
indexer = rl.Index()
indexer.add(Block('given_name'))
indexer.add(Block('surname'))
indexer.add(Block('soc_sec_id'))
candidate_links = indexer.index(dfA, dfB)
print(len(candidate_links), 'candidate links between dataset A and B')

Build index...
INFO:recordlinkage:indexing - initialize Index class
INFO:recordlinkage:indexing [1/?] - time: 0.89s - pairs: 160170/25000000 - rr: 0.99359
INFO:recordlinkage:indexing [1/?] - time: 0.89s - pairs_total: 160170/25000000 - rr_total: 0.99359
160170 candidate links between dataset A and B


In [7]:
# start comparing
print('Start comparing...')
comparer = rl.Compare()
comparer.add(Exact('given_name', 'given_name', label='given_name'))
comparer.add(String('surname', 'surname', method='jarowinkler',
                    threshold=0.85, label='surname'))
comparer.add(Exact('date_of_birth', 'date_of_birth', label='date_of_birth'))
comparer.add(Exact('suburb', 'suburb', label='suburb'))
comparer.add(Exact('state', 'state', label='state'))
comparer.add(String('address_1', 'address_1', threshold=0.85,
                    label='address_1'))
comparer.add(String('address_2', 'address_2', threshold=0.85,
                    label='address_2'))
features = comparer.compute(candidate_links, dfA, dfB)

print('feature shape', features.shape)

Start comparing...
INFO:recordlinkage:comparing - initialize Compare class
INFO:recordlinkage:comparing [1/?] - time: 2.94s - pairs: 160170
INFO:recordlinkage:comparing [1/?] - time: 2.94s - pairs_total: 160170
feature shape (160170, 7)


In [8]:
# Initialise the NaiveBayesClassifier.
cl = rl.SVMClassifier()
cl.fit(features, true_links)

INFO:recordlinkage:Classification - start training SVMClassifier
INFO:recordlinkage:Classification - training computation time: ~0.37s


In [9]:
# evaluate the model
links = cl.predict(features)
print("Predicted number of links:", len(links))

INFO:recordlinkage:Classification - predict matches and non-matches
Predicted number of links: 4928


In [10]:
cm = rl.confusion_matrix(true_links, links, len(candidate_links))
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[  4916     84]
 [    12 155158]]


In [11]:
# compute the F-score for this classification
fscore = rl.fscore(cm)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.9903303787268332
recall 0.9832
precision 0.997564935064935
